In [44]:
import pickle
import requests
import tenseal as ts
from train_utils import load_dataset, convert_to_binary
from sklearn.model_selection import train_test_split
import numpy as np
from cc_he_utils import json_serialize, json_deserialize, load_test_data_np, load_test_data_df, setup_tenseal_context
from ppxgboost import PPBooster
from ppxgboost import PaillierAPI as paillier

In [12]:
client_files_path = './client-files/'

In [6]:
dataset_name = 'ulb'

In [8]:
ulb_x, ulb_y = load_dataset(dataset_name)

ulb dataset loaded ✅..


In [9]:
_, ulb_xtest, _, ulb_ytest = train_test_split(ulb_x, ulb_y, test_size=0.2, shuffle=False)

In [2]:
# xtest_np, ytest_np = load_test_data_np()
# xtest_df, ytest_df = load_test_data_df()

In [3]:
# context = setup_tenseal_context()

# def make_nn_query():
#     enc_input = ts.ckks_vector(context, xtest_np[0])
#     request_data = {
#         'input': json_serialize(enc_input.serialize()),
#         'context': json_serialize(context.serialize()),
#     }
#     response = requests.post('http://134.226.86.101:8000/nn', json=request_data, timeout=10).json()
#     enc_x = ts.ckks_vector_from(context, json_deserialize(response['result']))
#     return enc_x.decrypt()

In [52]:
# prf_key = pickle.load(open('prf_key.pt', "rb"))
# encrypter = pickle.load(open('encrypter.pt', "rb"))
# min_max = pickle.load(open('min_max.pt', "rb"))
# private_key = pickle.load(open('private_key.pt', "rb"))
(column_hash_key, order_preserving_key, paillier_private_key, min_max) = pickle.load(open(client_files_path+'encrypted-ulb-xgboost-keys.pt', 'rb'))

def make_xgboost_query(plaintext_input, expected_output):
    encrypted_input = plaintext_input.copy()
    ppbooster.enc_input_vector(column_hash_key, order_preserving_key, set(plaintext_input.columns), encrypted_input, PPBooster.MetaData(min_max))
    request_data = {
        'input': json_serialize(encrypted_input)
    }
    response = requests.post('http://134.226.86.101:8000/xgboost/ulb', json=request_data, timeout=10).json()
    encrypted_result = json_deserialize(response['result'])
    decrypted_result = np.array([paillier.decrypt(paillier_private_key, enc_res) for enc_res in encrypted_result])
    is_expected = np.array_equal(convert_to_binary(decrypted_result), expected_output)
    print(is_expected)
#     return paillier.decrypt(paillier_private_key, encrypted_result)

In [53]:
# %time make_nn_query()

In [54]:
%time make_xgboost_query(ulb_xtest.head(1), ulb_ytest.head(1))

True
CPU times: user 346 ms, sys: 3.04 ms, total: 349 ms
Wall time: 359 ms


In [64]:
np.random.randint(0, len(ulb_xtest))

22230